# Example (multstars) Package Tutorial

This package will focus on modeling calculations using numpy ufuncs and doing posterior calculations. 

In [2]:
# Import packages
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime
import emcee
import corner
import matplotlib as mpl 
import corner 
import progressbar
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MaxNLocator
import cycler as cycler
import datetime
import pandas as pd

mpl.rcParams['figure.figsize'] = 12,8 
mpl.rcParams.update({'font.size': 14})

We'll set up a function to compute the predictions for $M_1$.  From Gregory eq. 3.27, model 1 predicts the spectral line will have a Gaussian line shape:
$$T f_i \equiv T \exp\left\{\frac{-(\nu_i-\nu_0)^2}{2\sigma_L^2}\right\}$$


In [3]:
def gaussian_line(channel, strength, center, width):
    """
    Gaussian lineshape
    
    Parameters
    ----------
    channel : float or ndarray
        channel number
    strength : float
        strength of signal (parameter T in model)
    center : float
        center of line (parameter nu_0 in model)
    width : float
        width of Gaussian line (parameter sigma_L in model)        
    """
    return strength * np.exp(-.5*(channel - center)**2 / width**2)

In [4]:
#let's evaluate the model at a single channel
print(gaussian_line(35, 1, 37, 1))

0.1353352832366127


### Doing posterior calculations
We'll make functions to evaluate both of these priors.  The functions will come in handy when we evaluate the posterior probability.  In both cases, we write docstrings so that it's easy for us (or someone else) to see what the function does.

- Uniform prior
    $$p(T|M_1,I) = \frac{1}{T_{\mathrm{max}}-T_{\mathrm{min}}}$$ 

- Jeffreys prior
    $$p(T|M_1,I) = \frac{1}{T \,\mathrm{ln}(T_{\mathrm{max}}/T_{\mathrm{min}})}$$

In [7]:
def uniform_prior(x, lower, upper):
    """
    Returns the value of the uniform prior
    
    Parameters
    ----------
    x : float or ndarray
        value of the parameter
    lower, upper : float
        values of the upper and lower bounds of the prior
    
    Returns
    -------
    float or ndarray : 
        the value of the prior
    """
    x = np.array(x)
    normalization = (upper - lower)
    out = np.zeros(x.shape) + ((x >= lower) & (x <= upper)) * 1./normalization
    return out

def jeffreys_prior(x, lower, upper):
    """
    Returns the value of the Jeffreys prior
    
    Parameters
    ----------
    x : float or ndarray
        value of the parameter
    lower, upper : float
        values of the upper and lower bounds of the prior
    
    Returns
    -------
    float or ndarray : 
        the value of the prior   
    """
    x = np.array(x)
    normalization = np.log(upper / lower)
    out = np.zeros(x.shape) + ((x >= lower) & (x <= upper)) * 1./(x * normalization)
    return out

In [9]:
#let's get the priors started by defining bounds,
prior1 = uniform_prior(0, 10,50)
prior2 = jeffreys_prior(1e-5, 1e5,5e5)